In [ ]:
!pip install gliner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.

In [ ]:
from datasets import load_dataset, Dataset
train_data = load_dataset(path='json', data_files='./KIND_train.json')['train']
validation_data = load_dataset(path='json', data_files='./KIND_validation.json')['train']

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(train_data[0])
print(validation_data[0])

{'id': 'WN:train:0', 'tokens': ['In', 'fiamme', "l'", 'Istituto', 'Lama', 'Tzong', 'Khapa', ',', 'monastero', 'buddhista'], 'labels': ['O', 'O', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 'O']}
{'id': 'WN:validation:0', 'tokens': ['Ricercatori', 'britannici', 'scoprono', 'il', 'primo', 'antibiotico', 'ricavato', 'dagli', 'insetti'], 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [ ]:
def convert_bio_to_entity_spans(entry, mapping):
    """
    Converts a dataset entry with BIO labels to the format expected by extract_entity_spans function.

    Args:
    - entry: A dictionary with keys 'id', 'tokens', 'labels'.
    - mapping: A dictionary mapping short entity types to their full forms.

    Returns:
    - A dictionary with keys 'tokenized_text', 'ner', and 'negative'.
    """
    tokens = entry['tokens']
    labels = entry['labels']

    entity_spans = []
    tokenized_text = tokens
    negative = []
    current_entity = None
    current_span = []

    for i, (token, label) in enumerate(zip(tokens, labels)):
        if label.startswith('B-'):
            if current_entity and current_span:
                entity_spans.append((current_span[0], current_span[-1], mapping.get(current_entity, current_entity)))
            current_entity = label[2:]
            current_span = [i]
        elif label.startswith('I-') and current_entity == label[2:]:
            current_span.append(i)
        else:
            if current_entity and current_span:
                entity_spans.append((current_span[0], current_span[-1], mapping.get(current_entity, current_entity)))
            current_entity = None
            current_span = []

    if current_entity and current_span:
        entity_spans.append((current_span[0], current_span[-1], mapping.get(current_entity, current_entity)))

    entity_types_in_text = {mapping.get(entity, entity) for _, _, entity in entity_spans}
    all_entity_types = {mapping.get(label[2:], label[2:]) for label in labels if label.startswith(('B-', 'I-'))}
    negative = list(all_entity_types - entity_types_in_text)

    #print(entity_spans)
    return {"tokenized_text": tokenized_text, "ner": entity_spans, "negative": negative}

# Example usage
"""
entry = {
    "id": 1,
    "tokens": ["OpenAI", "develops", "artificial", "intelligence", "in", "San", "Francisco"],
    "labels": ["B-ORG", "O", "B-FIELD", "I-FIELD", "O", "B-LOC", "I-LOC"]
}
"""
entry = train_data[0]

entity_mapping = {
    "PER": "persona",
    "ORG": "organizzazione",
    "LOC": "luogo"
}

output = convert_bio_to_entity_spans(entry, entity_mapping)
print(output)

{'tokenized_text': ['In', 'fiamme', "l'", 'Istituto', 'Lama', 'Tzong', 'Khapa', ',', 'monastero', 'buddhista'], 'ner': [(3, 6, 'luogo')], 'negative': []}


In [ ]:
import json
train_data_GLiNER_format = []
for sample in train_data:
    converted_sample = convert_bio_to_entity_spans(sample, entity_mapping)
    train_data_GLiNER_format.append(converted_sample)

with open("./KIND_train_GLiNER_format.json", 'w') as f:
    json.dump(train_data_GLiNER_format, f)

validation_data_GLiNER_format = []
for sample in validation_data:
    converted_sample = convert_bio_to_entity_spans(sample, entity_mapping)
    validation_data_GLiNER_format.append(converted_sample)

with open("./KIND_validation_GLiNER_format.json", 'w') as f:
    json.dump(validation_data_GLiNER_format, f)

In [ ]:
for sample in train_data_GLiNER_format:
  if len(sample['tokenized_text']) < 5:
    print(sample)

{'tokenized_text': ['Cagliari', ':'], 'ner': [(0, 0, 'luogo')], 'negative': []}
{'tokenized_text': ['Significato', "dell'", 'esposizione'], 'ner': [], 'negative': []}
{'tokenized_text': ['I', 'Paesi', 'interessati'], 'ner': [], 'negative': []}
{'tokenized_text': ['Tutti', 'a', 'bordo', '!'], 'ner': [], 'negative': []}
{'tokenized_text': ['di', 'giorno', 'e', '45'], 'ner': [], 'negative': []}
{'tokenized_text': ['di', 'notte', '.', '»'], 'ner': [], 'negative': []}
{'tokenized_text': ['Paolo', 'Nespoli', '.'], 'ner': [(0, 1, 'persona')], 'negative': []}
{'tokenized_text': ['Scaramella', 'resta', 'in', 'carcere'], 'ner': [(0, 0, 'persona')], 'negative': []}
{'tokenized_text': ['numerosi', 'feriti'], 'ner': [], 'negative': []}
{'tokenized_text': ['entrambi', 'morti'], 'ner': [], 'negative': []}
{'tokenized_text': ['In', 'Europa'], 'ner': [(1, 1, 'luogo')], 'negative': []}
{'tokenized_text': ['In', 'Albania'], 'ner': [(1, 1, 'luogo')], 'negative': []}
{'tokenized_text': ['In', 'Bulgaria'], 

In [ ]:
pip install accelerate -U

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
import argparse
import random
import json

from transformers import AutoTokenizer
import torch

from gliner import GLiNERConfig, GLiNER
from gliner.training import Trainer, TrainingArguments
from gliner.data_processing.collator import DataCollatorWithPadding
from gliner.utils import load_config_as_namespace
from gliner.data_processing import WordsSplitter, GLiNERDataset

from transformers import EarlyStoppingCallback


device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

config = load_config_as_namespace("./config.yaml")
config.log_dir = "GLINER_output"

model_config = GLiNERConfig(**vars(config))

with open(config.train_data, 'r') as f:
    train_data = json.load(f)

with open(config.val_data_dir, 'r') as f:
    test_data = json.load(f)

train_data = [sample for sample in train_data if len(sample['tokenized_text']) > 10]
#test_data = [sample for sample in test_data if len(sample['tokenized_text']) > 10]
test_data = train_data[int(len(train_data)*0.9):]


tokenizer = AutoTokenizer.from_pretrained(model_config.model_name)
model_config.class_token_index=len(tokenizer)
tokenizer.add_tokens([model_config.ent_token, model_config.sep_token])
model_config.vocab_size = len(tokenizer)

words_splitter = WordsSplitter(model_config.words_splitter_type)

train_dataset = GLiNERDataset(train_data, model_config, tokenizer, words_splitter)
test_dataset = GLiNERDataset(test_data, model_config, tokenizer, words_splitter)

print(test_dataset)

#print(train_dataset[0])
#print(test_dataset[0])

data_collator = DataCollatorWithPadding(model_config)

model = GLiNER(model_config, tokenizer=tokenizer, words_splitter=words_splitter)
model.resize_token_embeddings([model_config.ent_token, model_config.sep_token],
                              set_class_token_index = False,
                              add_tokens_to_tokenizer=False)

training_args = TrainingArguments(
    output_dir=config.log_dir,
    learning_rate=float(config.lr_encoder),
    weight_decay=float(config.weight_decay_encoder),
    others_lr=float(config.lr_others),
    others_weight_decay=float(config.weight_decay_other),
    lr_scheduler_type=config.scheduler_type,
    warmup_ratio=config.warmup_ratio,
    per_device_train_batch_size=config.train_batch_size,
    per_device_eval_batch_size=config.train_batch_size,
    max_grad_norm=config.max_grad_norm,
    #max_steps=config.num_steps,
    num_train_epochs=3,
    #gradient_accumulation_steps=config.gradient_accumulation_steps,
    #evaluation_strategy="steps",
    #eval_steps=config.eval_every,
    save_steps = config.eval_every,
    save_total_limit=config.save_total_limit,
    dataloader_num_workers = 8,
    use_cpu = False,
    report_to="none",
    #load_best_model_at_end=True
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    #eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.


100%|██████████| 8200/8200 [00:00<00:00, 522297.54it/s]


Total number of entity classes:  3


100%|██████████| 820/820 [00:00<00:00, 565679.16it/s]

Total number of entity classes:  3


...

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Skipping iteration due to error: CUDA out of memory. Tried to allocate 194.00 MiB. GPU 
Skipping iteration due to error: CUDA out of memory. Tried to allocate 36.00 MiB. GPU 
Skipping iteration due to error: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 
Skipping iteration due to error: CUDA out of memory. Tried to allocate 40.00 MiB. GPU 
Skipping iteration due to error: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Skipping iteration due to error: CUDA out of memory. Tried to allocate 194.00 MiB. GPU 
Skipping iteration due to error: CUDA out of memory. Tried to allocate 46.00 MiB. GPU 
Skipping iteration due to error: CUDA out of memory. Tried to allocate 90.00 MiB. GPU 
Skipping iteration due to error: CUDA out of memory. Tried to allocate 68.00 MiB. GPU 


TrainOutput(global_step=3075, training_loss=36.708222919247014, metrics={'train_runtime': 2422.2726, 'train_samples_per_second': 10.156, 'train_steps_per_second': 1.269, 'total_flos': 0.0, 'train_loss': 36.708222919247014, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./GLiNER_WN_model_3")

In [ ]:
!pip show accelerate


Name: accelerate
Version: 0.32.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [ ]:
from datasets import load_dataset, Dataset
import torch
torch.cuda.empty_cache()

In [ ]:
#test_data = load_dataset(path='json', data_files='./kind_test.json')['train']
test_data = load_dataset(path='json', data_files='./MultinerdIT_test.json')['train']

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(test_data)

Dataset({
    features: ['tokens', 'labels', 'id'],
    num_rows: 18217
})


In [ ]:
from gliner import GLiNER

model_name = 'urchade/gliner_multi-v2.1'
#model_name = "DeepMount00/universal_ner_ita"
model = GLiNER.from_pretrained(model_name, map_location='cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/4.77k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
import json
import os
with open(os.path.join('./extended_labels_map.jsonl')) as fp:
  extended_labels_map = json.load(fp)

In [ ]:
#labels_option = ['persona', 'organizzazione', 'luogo']
labels_option = list(extended_labels_map.values())
print(labels_option)

['persona', 'organizzazione', 'luogo', 'animale', 'entità biologica', 'corpo celeste', 'malattia', 'evento', 'cibo', 'strumento', 'media', 'entità mitologica', 'pianta', 'tempo', 'veicolo']


In [ ]:
predictions = []
batch_size = 32
test_data = test_data.to_list()
for i in range(0, len(test_data), batch_size):
  input_batch = []
  for chunk_sample in test_data[i:i+batch_size]:
    input = ' '.join(chunk_sample['tokens'])
    input_batch.append(input)
  batch_predictions = model.batch_predict_entities(input_batch, labels_option)
  for chunk_sample, pred in zip(test_data[i:i+batch_size], batch_predictions):
    predictions.append({
      'id': chunk_sample['id'],
      'tokens': chunk_sample['tokens'],
      'labels': chunk_sample['labels'],
      'prediction': pred
  })

In [ ]:
for pred in predictions:
  print(pred)

Streaming output truncated to the last 5000 lines.
{'id': 'it:test:13217', 'tokens': ['Originariamente', 'si', 'pensava', 'che', 'questo', 'animale', 'fosse', 'uno', 'stretto', 'parente', 'dell', '"', 'Archaeopteryx', '"', '.'], 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ANIM', 'O', 'O'], 'prediction': [{'start': 79, 'end': 92, 'text': 'Archaeopteryx', 'label': 'entità biologica', 'score': 0.8055433630943298}]}
{'id': 'it:test:13218', 'tokens': ['Scoperto', 'nel', '1914', ',', 'presenta', "un'", 'orbita', 'caratterizzata', 'da', 'un', 'semiasse', 'maggiore', 'pari', 'a', '3,1272181', '"', 'UA', 'e', 'da', "un'", 'eccentricità', 'di', '0,2966770', ',', 'inclinata', 'di', '5,41684', '°', 'rispetto', "all'", 'eclittica', '.'], 'labels': ['O', 'O', 'B-TIME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'prediction': []}
{'id': 'it:test:13219', 'tokens': ['Frie

In [ ]:
predictions_Dataset = Dataset.from_list(predictions)
#predictions_Dataset.to_json("./universal_ner_ita_KIND_preds.jsonl")
#predictions_Dataset.to_json(f"./{model_name}_MultinerdIT_preds.jsonl")

predictions_Dataset.to_json("./GLiNER_WN_model_3_MultinerdIT_preds.jsonl")

Creating json from Arrow format:   0%|          | 0/19 [00:00<?, ?ba/s]

8787271